In [2]:
import numpy as np
import pandas as pd
import re
import pickle
from pprint import pprint


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import json
data = json.loads('{"one" : "1", "two" : "2", "three" : "3"}')

from bson import json_util

from mongoengine import *

In [3]:
conn = connect("local",host="mongodb://192.168.0.5:27017",alias='default')

In [4]:
disconnect('mongo://192.168.100.13:27017')

In [5]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

restautant_id = 'BNGHSR0001'

final_list_json=[]

In [6]:

for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['foodlist']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['foodlist'].append({})
            final_list_json[n]['sub_category'][p]['foodlist'][q]['name']= food_dict[1]['dish_name']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['price']=food_dict[1]['price']
        
        

KeyError: 'dish_name'

In [180]:
class FoodItem(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    price = StringField(required=True)
    food_id = StringField(required=True)

class SubCategory(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    sub_cat_id = StringField(required=True)
    foodlist= ListField(ReferenceField(FoodItem),required=True)
    
    def to_mymongo(self):
        data = self.to_mongo()
        for key, fooditem in enumerate(self.foodlist):
            data['foodlist'][key]=self.foodlist[key].to_mongo()
#             reply["to_users"] = {"User": {"username": user.username} }

        return data
    
    meta = {'allow_inheritance' : True}

class MainCategory(Document):
    name = StringField(required=True)
    description = StringField(required=True)
    main_cat_id = StringField(required=True)
    sub_category= ListField(ReferenceField(SubCategory),required=True)
    
    def to_mymongo(self):
        data = self.to_mongo()
        for key, sub_cat in enumerate(self.sub_category):
            data['sub_category'][key]=self.sub_category[key].to_mymongo()
#             reply["to_users"] = {"User": {"username": user.username} }

        return data


class Restaurant(Document):
    name = StringField(required=True)
    restaurant_id = StringField(required=True)
    unique_id = StringField(required=True)
    menu = ListField(ReferenceField(MainCategory),required=True)
    address = StringField()
    tables = StringField()
    servers = StringField()
    
    def to_json(self):
        data = self.to_mongo()
        for key, sub_cat in enumerate(self.menu):
            data['menu'][key]=self.menu[key].to_mymongo()
#             reply["to_users"] = {"User": {"username": user.username} }

        return json_util.dumps(data)
    
    meta = {'allow_inheritance' : True}

In [177]:
Restaurant.objects[0].to_json()

'{"_id": {"$oid": "5e7487d4f1dfa2dd7ec0c16d"}, "_cls": "Restaurant", "name": "House of Commons", "restaurant_id": "BNGHSR0001", "unique_id": "0000000001", "menu": [{"_id": {"$oid": "5e7487d2f1dfa2dd7ec0c128"}, "name": "BAR BITES", "description": "", "main_cat_id": "BNGHSR00010", "sub_categories": [{"_id": {"$oid": "5e7487d2f1dfa2dd7ec0c127"}, "_cls": "SubCategory", "name": "BAR BITES", "description": "", "sub_cat_id": "BNGHSR000100", "foodlist": [{"_id": {"$oid": "5e7487d1f1dfa2dd7ec0c120"}, "name": "NACHOS (Veg/Chicken)", "description": "Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream", "price": "190/230", "food_id": "BNGHSR0001000"}, {"_id": {"$oid": "5e7487d1f1dfa2dd7ec0c121"}, "name": "CHEESE MASALA PAPAD", "description": "Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese", "price": "160", "food_id": "BNGHSR0001001"}, {"_id": {"$oid": "5e7487d1f1dfa2dd7ec0c122"}, "name": "H.O.C SIGNATURE MASALA PEANUTS", "description": "A Favorful

In [178]:
pprint(json.loads(Restaurant.objects[0].to_json()))

{'_cls': 'Restaurant',
 '_id': {'$oid': '5e7487d4f1dfa2dd7ec0c16d'},
 'address': '',
 'menu': [{'_id': {'$oid': '5e7487d2f1dfa2dd7ec0c128'},
           'description': '',
           'main_cat_id': 'BNGHSR00010',
           'name': 'BAR BITES',
           'sub_categories': [{'_cls': 'SubCategory',
                               '_id': {'$oid': '5e7487d2f1dfa2dd7ec0c127'},
                               'description': '',
                               'foodlist': [{'_id': {'$oid': '5e7487d1f1dfa2dd7ec0c120'},
                                             'description': 'Crispy Corn '
                                                            'Tortillas Topped '
                                                            'With Spicy Baked '
                                                            'Bean Salsa and '
                                                            'Sour Cream',
                                             'food_id': 'BNGHSR0001000',
                          

In [191]:
FoodItem.drop_collection()
SubCategory.drop_collection()

MainCategory.drop_collection()
Restaurant.drop_collection()


main_cat_list = []
for p,main_cat in enumerate(final_list_json):
    sub_cat_list = []
    for q,sub_cat in enumerate(main_cat['sub_category']):
        food_list = []
        for r,food in enumerate(sub_cat['foodlist']):
            food_list.append(FoodItem(name=food['name'],description = food['description'],price=food['price'],food_id = restautant_id+str_2(p)+str_2(q)+str_2(r)).save())
        sub_cat_list.append(SubCategory(name=sub_cat['name'],description=sub_cat['description'],foodlist = food_list,sub_cat_id = restautant_id+str_2(p)+str_2(q)).save())
    main_cat_list.append(MainCategory(name=main_cat['name'],description = main_cat['description'],sub_category = sub_cat_list,main_cat_id = restautant_id+str_2(p)).save())
    
    
houseofcommons=Restaurant(name = 'House of Commons',restaurant_id='BNGHSR0001',unique_id = '0000000001',menu = main_cat_list,address='')
houseofcommons.save()

<Restaurant: Restaurant object>

In [187]:
def str_2(number):
    string = str(number)
    if(len(string)==1):
        return '0'+string
    else:
        return string

In [190]:
str_2(8)

'08'

# Pages

In [7]:
pages={}

In [8]:
pages[0]="""$BAR BITES

NACHOS (Veg/Chicken)

Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream 

190/230

CHEESE MASALA PAPAD

Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese

160

H.O.C SIGNATURE MASALA PEANUTS

A Favorful Version Of Regular Peanuts Tossed In A Variety of Spices 

160

SPICY TRICOLOUR CHAAT

Fresh Peas, Corn Kernels & Black Gram Tossed With Chillies & Sirka onions

180

CHATPATA CHANA CHAAT

A Delectable Blend Of Bengal Gram And A Tangy Mixture

160

TANDOORI CHICKEN CHAAT

Clay Oven Roasted Chicken Blended With Classic Spices

220

CHEESE DYNABITES

Classic Cheese Balls Served with New Orleans Dip

220

$SALADS

GREEN SALAD

Traditional Green Cuts With Mint Yoghurt Dip

170

HOC SALAD (Veg/Chicken)

Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing

180/210

WATERMELON & FETA

An All Time Classic - With Balsamic Glaze

210

CAESAR SALAD (Veg/Chicken)

Crunchy Ice Berg Lettuce, Parmesan With the Classic Caesar Dressing

190/230"""

In [9]:
pages[1]="""$VEG STARTERS

CHEESE & CORN TRIANGLE (new)

Shredded Cheese, Corn & Seasonings Wrapped In Wonton Sheet & Deep Fried

220

CHAMPIGNONS FARCIS (new)

Sauteed Button Mushroom Stuffed With Cheese & Herbs

230

CHILLI CHEESE CIGARA (new)

Deep Fried Rolls With Cheese, Chill & Herbs

220

HOC CHIP & DIP BASKET

Combination Of Onion Rings, Jalapeno Cheese Balls, Greek Potato Wedges, Habanero Green Chili Fries

390

FRENCH FRIES

French Fries Tossed With Hoc Signature Cajun Spice Seasoning

190

ONION RINGS

Extra Crispy Onion Rings With A Spicy Salsa

190

MUSHROOM LOADED FRIES

Cheesy French Fries Topped With Mushrooms And Herbs

230

GARLIC BREAD

Garlic Butter Flavoured Bread Topped With Cheese

190

CHEESE CORN TIKKI

American Com & Cottage Cheese Patty Served with Mint Chutney

220

CRISPY CHILLY MUSHROOM

Crispy Fried Mushrooms Tossed With Oriental Spices

220

CRISPY CHILLY BABY CORN

Crispy Fried Babycom Tossed With Oriental Spices

190

CRISPY CORN CHILLY PEPPER

Fred Undi Lightly Crispy & Seasoned With Lime Juice, Salt, Chilli & Pepper

240

HONEY CHILLY POTATO

Crispy Fried Potatoes Tossed With Chilli And Honey

190"""

In [10]:
pages[2]="""$VEG STARTERS

SESAME CHILLI PANEER

Cottage Cheese Cubes Tossed In A Sweet & Spicy Sauce, Finished With Sesame Seeds

270

PANDAN LEAF COTTAGE CHEESE

Tender Cottage Cheese Wrapped In Fragrant Pandan Leaves, With Chilli Coriander Sauce

280

MEZZE BASKET

A Combination Of Hummus, Mutabal & Labneh, Served With Pita Bread

340

$FROM THE CLAY OVEN

MAKHMALI PANEER TIKKA

Mouth Melting Paneer Cubes Marinated With A Cheese Curd Mixture, Cream And Mild Spices

270

ANGARI PANEER TIKKA

Clay Oven Roasted Spicy Morsels Of Cottage Cheese

270

TANDOORI STUFFED MUSHROOMS

Mushrooms Filled With A Blend Of Indian Spices And Tangy Stuffing Mix, Adding A Unique Taste On The Bite

250

TANDOORI VEG MOMOS

Soft And Juicy Vegetable Momos Marinated With Tandoori Masala And Char Grilled

220

$CHICKEN STARTERS

CHICKEN LOADED FRIES

Cheesy French Fries Topped With Chicken & Herbs

260

SHANG DONG CHICKEN LOLLY POP

Crispy Lolly pops Crunched To Perfection & Tossed in our Signature Shang Dong Drip

260"""

In [11]:
pages[3]="""$CHICKEN STARTERS

HOC SIGNATURE CHICKEN WINGS

Sweet Chilly / Spiced Szechuan

260

PEPPER GARLIC CHICKEN

Cubes Of Chicken Tossed With Garlic And Pepper In Oriental Style

260

GUNTUR JUNCTION CHICKEN

Chicken Cubes Tossed With Freshly Ground Red Spices And Finished In South Indian Style

270

ANDHRA CHILLI CHICKEN *VERY HOT*

All Time Favourite Andhra Style Chicken Cooked With Coriander And Green Chilli Paste And Fresh Green Chillies

270

LEMON PEPPER CHICKEN

Slice Chicken Wok Tossed, Finished With Pepper Garlic & Lime

260

JIANG CHILLI CHICKEN

Chicken Tossed With Shredded Ginger And Chilli In Jiang Style

260

CHICKEN GHEE ROAST

Chicken Roasted With A Fiery Red, Tangy And Spicy With The Flavour Of Ghee

280

MASALA FRY CHICKEN (new)

Classic Indian Style With Bone Deep fried Chicken Served With Mint Chutney

250

$FROM THE CLAY OVEN

JAIPURI MURGH KA SOOLA

Succulent Pieces Of Chicken Zapped With Red Spices, Cinnamon Flavour & Finished In Tandoor

270

TANDOORI MURGH (HALF/FULL)

Chicken Marinated In Aromatic Indian Spices & Then Cooked In Tandoor

250/480

TANDOORI POMFRET

Indian Spices Marinated Whole Pomfret Skewered And Finished In Clay Oven

590"""

In [12]:
pages[4]="""$FROM THE CLAY OVEN

FISH TIKKA KALIMIRCH

Fresh Cubes Of Basa Fish Marinated With Crushed Black Pepper & Indian Spices, Finished In Clay Oven

320

AMRITSAR FISH TIKKA

Spicy Fish Tikka Marinated In Traditional Spices And Char Grilled

320

KALMI KEBAB (3pcs)

Clay Oven Roasted Chicken Drumsticks

260

ANGARI MURGH TIKKA

Clay Oven Roasted Spicy Morsels Of Chicken

260

TANDOORI LOLLIPOPS

Chicken Lollipops Marinated With Aromatic Indian Spices And Cooked In The Tandoor

270

MURGH MALAI TIKKA

Clay Oven Roasted Creamy Chunks Of Chicken

260

MURGH KASTURI KEBAB

Boneless Charcoal Smoked Chicken Tikka With Kasturi Methi Flavoured

260

TANDOORI CHICKEN MOMOS

Soft And Juicy Chicken Momos Marinated With Tandoori Masala And Char Grilled

260

TANDOORI KING PRAWNS

King Prawns Marinated With Yogurt, Lime Juice, Bhatti Spices And Slowly Cooked In Tandoor

499

$LAMB STARTERS

ANDHRA LAMB PEPPER FRY

Tender Mutton Pieces Tossed In South Indian Spices

350

MUTTON GHEE ROAST

Mutton Roasted With A Fiery Red And Spicy With The Flavour Of Ghee

370"""

In [13]:
pages[5]="""$LAMB STARTERS

LAHORI SEEKH KEBAB

Succulent Minced Lamb Kebabs In The True Lahori Street Food Style

370

RAILWAY MUTTON DRY

With Bone Mutton Cooked In Rich Indian Spices, As Served In The Railway Canteen Of The British Raj

350

$EGG STARTERS

EGG BHURJI

Desi Style Scrambled Eggs

170

SPICY MASALA OMELETTE

Slow Cooked Egg Omlet With Masala

180

EGG CHILLI 

Spicy Boiled Egg With Chilli Fry

210

$FROM THE GRILL

PERI PERI PANEER/CHICKEN

Peri Peri Basted Paneer / Chicken Cubes Served With Tzatziki Dip

260/280

THAI GRILLED FISH IN BANANA LEAF

Delicately Grilled Fish With Basil Flavour And Served Hot

330

HARISSA GRILLED FISH

Fish Marinated In A Spicy Tunisian Harissa Sauce And Grilled To Perfection

330

THAI GRILLED CHICKEN

Boneless Chicken Marinated With Thai Spices And Grilled

270"""

In [14]:
pages[6]="""$SEAFOOD STARTERS

PAN FRIED CHILLY FISH

Pan Fried Fish Tossed With Shredded, Onion, Capsicum And Green Chilli

330

THAI PEPPER GARLIC PRAWNS

Prawns Prepared In Thai Style With A Flavour Of Lemon Grass

380

PRAWNS IN RCP SAUCE

Wok Tossed Prawns Cooked With Roasted Chilli Paste And Seasonings

380

H.O.C FISH FINGERS

Classic Crumb fried Served with Tartar Sauce

330

ASIAN STYLE JUMBO PRAWNS

Tsing Hol Style / Singapore Chilli Garlic Style

499

$SANDWICHES/BURGERS

TOASTED VEG CLUB SANDWICH

Double Decker Layered With Lettuce, Cucumber, Cheese, Tomato And Served With Fries

240

TOASTED CHICKEN CLUB SANDWICH

Double Decker Layered With Lettuce, Cheese, Chicken, Fried Egg And Served With Fries

270

CLASSIC BURGERS (VEG / CHICKEN / LAMB)

Tender And Juicy Veg, Chicken Or Lamb Patties, Grilled And Served Fries With

250/280/300

HOC BURGER- THE BIG ONE

Double Chicken, Double Cheese And Topped With A Sunny Side Up Fried Egg, Served With Fries

350

$THE GRILL SIZZLERS

MIXED VEGETABLE SIZZLER

Grilled Vegetable Patties, Cottage Cheese And English Vegetables, Served With French Fries

350"""

In [15]:
pages[7]="""$THE GRILL SIZZLERS

ROSEMARY SCENTED COTTAGE CHEESE STEAKS

Cottage Cheese Steaks Wth Bell Peppers, Sage Pilaf With Grilled Vegetables & Jalapeno Tomato Coulis

380

CHICKEN CAFREAL

Authentic Goan Herb Marinated Chicken, Pan Fried, Stewed And Served With Potato Wedges & Sauteed Vegetables

390

CHICKEN SIZZLER

Chicken with red wine sauce, served with saute vegetable and french fries

380

TEQUILA LIME GRILLED CHICKEN

Flesh Flamed Tequila Marinated Lime & Chilli Chicken With Herbed Country Mash, Garden Grilled Vegetables Cumin Coffee Scented Jus

400

GRILLED FISH

Grilled Fish Fillet, With Lemon butter sauce Served With Herb Rice & saute vegetables

400

CAPTAIN MORGAN FLAMED LAMB

Rum Infused Caribbean Lamb & Pine Nut Patty With A Centred Egg, Charred Peppers, Jalapeno Aloll, Smoked Onion Marmalade, English Cheddar Melt, French Fries With A Sage & Onion

440

$PASTAS

BAKED CHEESE MACARONI

All Time Favourite In A Cheesy Cream Sauce And Baked in The Oven

300

SPAGHETTI AGLIO OLIO (VEG/ CHICKEN / SHRIMPS)

NA

300/340/380

FIERY PENNE PASTA (VEG/ CHICKEN / SHRIMPS)

NA

300/340/380

SPICY CAJUN LINGUINE ALFREDO (VEG/ CHICKEN / SHRIMPS)

NA

300/340/380

PENNE WITH PESTO (VEG/ CHICKEN / SHRIMPS)

NA

300/340/380"""

In [16]:
pages[8]="""$PIZZAS

MARGHERITA

An All-Time Favourite Of Pomodoro, Water Buffalo Mozzarella And Basil Chiffonade

290

FUNGHI

Shitake, Oyster & Button Mushrooms With Shallots, Chives, Charmed Peppers And White Truffle Oil

300

PANEER TIKKA & CHARRED PEPPERS

Tandoori Paneer Mirch Tikka With Green Bell Peppers, Onion roundels, Spring Greens & Mint Coulis

350

MEDITERRANEAN GARDEN

Organic Greens With Imported FETA Cheese, Bell Peppers, Zucchini, Asparagus Spears, Artichokes & Sun Dried Tomatoes

360

TANDOORI MURGH TIKKA WITH KALAMATA OLIVES

Chicken Tikka With Shallots, Capers, Charred Peppers, Kalamata Olives & Sweet Chilli Treacle

380

BARBEQUED GERMAN CHICKEN SAUSAGES

BBQ Glazed Chicken Sausages With Caramelized Shallots, Bell Peppers, Jalapenos & Capers

390

MUTTON MASALA KEEMA PIZZA

Minced Mutton Cooked In Indian Spices & Herbs Topped With Chillies & Assorted Bell peppers

390

FRUITS OF THE SEA

Fruits Of The Sea - Calamari, Shrimps, Sea Bream Dices Served With Maple Butter & Fresh Red Chile

390

MIXED MEAT MAGIC

Age Old English Cheddar Topped With Roast Chicken Slvers, Apple Glazed Chicken Ham, Lamb Mince, Calamari & Shrimps

420"""

In [17]:
pages[9]="""$MAIN COURSE

FISH AND CHIPS

Crumb Fried Fish In Hoc Signature Marination, Served With French Fries

350

CHICKEN BASKET

Crumb Fried Chicken Supreme In Hoc Signature Marination, Served With French Fries

350

THAI CURRY [Red/Green] 

Basil Flavoured Thal Curry/bird-eye Chillies Flavoured Thai Curry Cooked With Garden Fresh Vegetables/chicken/prawns, Served With Steamed Rice

250/280/300

CHINESE CHOPSUEY

Vegetables/chicken Tossed In Celery Sauce, Served On A Bed Of Crispy Noodles

280/300

AMERICAN CHOPSUEY

Vegetables/chicken Tossed In A Sweet & Sour Sauce, Served On A Bed Of Crispy Noodles

280/300 

EXOTIC VEGETABLES WITH SAUCE OF YOUR CHOICE

Hot Garlic, Chili Basil, Sichuan

250

CHICKEN WITH SAUCE OF YOUR CHOICE

Hunan, Sichuan, Black Pepper

280

BASA FISH/PRAWNS WITH CHOICE OF YOUR CHOICE

Ginger Chilli, Sichuan, Pepper Garlic

270/300

TIKKA MASALA GRAVY

Paneer/chicken Cooked In A Spicy Onion Tomato Base Gravy

250/280

PALAK PANEER

Cottage Cheese Cooked In Spinach Puree, Tempered Cumin And Ginger Garlic Paste

260

DAL MAKHANI

Slow Cooked Lentils Finished With White Butter

240"""

In [18]:
pages[10]="""$MAIN COURSE

MAKHANI GRAVY

Paneer/Chicken Cooked In A Creamy Tomato Base Gravy

250/280

KHUMB HARA PYAZA

Vintage Mushrooms & Spring Onions Cooked To Perfection

270

DAL TADKA

Yellow Lentil Tempered With Garlic And Cumin Seeds, Punished With Chopped Onion And Tomato

190

BUTTER CHICKEN

All-Time Favorite

280

KADAI VEG/CHICKEN

Mix Veg/Chicken Pleces Tossed With Onion, Capsicum And Garnished With Onion Tomato Gravy

250/270 

MURGH METHI KHAS

Tender Chicken Cooked In Cream Based Onion And Tomato Gravy With Aromatic Spices And Flavor

280

DUM KA GHOST

Slow Cooked Mutton In A Spicy Onion & Tomato Gravy

330

$RICE & NOODLES

STIR FRIED RICE

(veg/egg/chicken/prawns)

200/220/250/280

STIR FRIED NOODLES

(veg/egg/chicken/prawns)

200/220/250/280

SICHUAN FRIED RICE

(veg/egg/chicken/prawns)

200/220/250/280

SICHUAN FRIED NOODLES

(veg/egg/chicken/prawns)

200/220/250/280

CHILLY GARLIC NOODLES

(veg/egg/chiden/prawns)

230/250/270/290"""

In [19]:
pages[11]="""$ACCOMPANIMENTS

JEERA RICE

NA

160

STEAMED RICE

NA

130

ROTI PLAIN/BUTTER

NA

35/40

KULCHA PLAIN/BUTTER

NA

35/40

NAAN PLAIN/BUTTER

NA

40/45

GARLIC NAAN

NA

50

LACCHA PARATHA

NA

45

$DESSERTS

GULAB JAMUN

Cardamom Flavoured Milk Dumplings Soaked In Sugar Syrup

90

WALNUT BROWNIE

Warm Chocolate Brownie Served With Vanilla ice cream

170

CARAMEL CUSTARD

Custard Dessert With A Layer Of Soft Caramel Glazed On Top

150

CHOICE OF ICE CREAMS-2 Scoops

Ask for the Flavours

90"""

# Consolidating

In [20]:
half_the_menu=[]

for i in range(12):
    half_the_menu.append(pages[i])

half_menu_text="\n\n".join(half_the_menu)

In [21]:
structured_menu = pd.DataFrame(columns=['sub_category','name','description','price'])

In [22]:
description=dish_name=price=subcat=''
counter=0
for line in half_menu_text.split("\n\n"):
    if(re.search("^[\$]",line)):
        subcat = line[1:].strip()
    else:
        if(counter%3==0):
            dish_name = line.strip()
        elif(counter%3==1):
            description = line.strip()
        elif(counter%3==2):
            price = line.strip()
            structured_menu.loc[structured_menu.shape[0]]=[subcat,dish_name,description,price]
        counter+=1

In [23]:
if(0):
    pickle.dump(structured_menu,open('half_menu.pkl','wb'))

    str_menu=pickle.load(open('half_menu.pkl','rb'))

In [24]:
pickle.dump(structured_menu,open('full_menu.pkl','wb'))

In [25]:
str_menu=pickle.load(open('full_menu.pkl','rb'))

In [26]:
str_menu

,sub_category,name,description,price
0,BAR BITES,NACHOS (Veg/Chicken),Crispy Corn Tortillas Topped With Spicy Baked ...,190/230
1,BAR BITES,CHEESE MASALA PAPAD,Crispy fried Mini Papads Topped With A Tangy M...,160
2,BAR BITES,H.O.C SIGNATURE MASALA PEANUTS,A Favorful Version Of Regular Peanuts Tossed I...,160
3,BAR BITES,SPICY TRICOLOUR CHAAT,"Fresh Peas, Corn Kernels & Black Gram Tossed W...",180
4,BAR BITES,CHATPATA CHANA CHAAT,A Delectable Blend Of Bengal Gram And A Tangy ...,160
5,BAR BITES,TANDOORI CHICKEN CHAAT,Clay Oven Roasted Chicken Blended With Classic...,220
6,BAR BITES,CHEESE DYNABITES,Classic Cheese Balls Served with New Orleans Dip,220
7,SALADS,GREEN SALAD,Traditional Green Cuts With Mint Yoghurt Dip,170
8,SALADS,HOC SALAD (Veg/Chicken),"Melange Of Crunchy Lettuce, Bell Peppers And O...",180/210
9,SALADS,WATERMELON & FETA,An All Time Classic - With Balsamic Glaze,210


In [27]:
for sub_cat in str_menu.sub_category.to_list():
    try:
        if(main_category[sub_cat]):
            pass
    except:
        main_category[sub_cat]='REST'

NameError: name 'main_category' is not defined

In [28]:
main_category={"BAR BITES":"BAR BITES","SALADS":"SALADS","VEG STARTERS":"VEG STARTERS","CHICKEN STARTERS":"NON-VEG STARTERS","LAMB STARTERS":"NON-VEG STARTERS","EGG STARTERS":"NON-VEG STARTERS","FROM THE GRILL":"NON-VEG STARTERS","FROM THE CLAY OVEN":"CHANGE"}

In [55]:
str_menu.loc[:,"main_category"]=str_menu['sub_category'].apply(lambda x: main_category[x])

In [56]:
str_menu.loc[27,"main_category"]="VEG STARTERS"

str_menu.loc[28,"main_category"]="VEG STARTERS"

str_menu.loc[29,"main_category"]="VEG STARTERS"

str_menu.loc[30,"main_category"]="VEG STARTERS"

In [57]:
str_menu.loc[str_menu['main_category']=='CHANGE',"main_category"]="NON-VEG STARTERS"

In [58]:
str_menu.columns

Index(['sub_category', 'name', 'description', 'price', 'main_category'], dtype='object')

In [60]:
str_menu=str_menu[['main_category','sub_category', 'name', 'description', 'price']]

In [61]:
pickle.dump(str_menu,open('full_menu_main.pkl','wb'))

In [62]:
str_menu=pickle.load(open('full_menu_main.pkl','rb'))

In [63]:
from flask import Flask,jsonify, request, send_from_directory, url_for
app = Flask(__name__)
import pickle


In [64]:
final_json={}

In [15]:

for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_json["cat"+str(n)]={}
    final_json["cat"+str(n)]['name']=main_cat_name
    final_json["cat"+str(n)]['description'] = ''
    final_json["cat"+str(n)]['sub_category'] = {}
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]={}
        final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]['name']=sub_cat_name
        final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]['description']=''
        final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]['priority']=''
        final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]['foodlist']={}
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]['foodlist']['food'+str(q)]= {}
            final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]['foodlist']['food'+str(q)]['name']= food_dict[1]['dish_name']
            final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]['foodlist']['food'+str(q)]['description']=food_dict[1]['description']
            final_json['cat'+str(n)]['sub_category']["sub_cat"+str(p)]['foodlist']['food'+str(q)]['price']=food_dict[1]['price']
        
        

In [68]:
final_list_json=[]

In [69]:

for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['foodlist']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['foodlist'].append({})
            final_list_json[n]['sub_category'][p]['foodlist'][q]['name']= food_dict[1]['name']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['price']=food_dict[1]['price']
        
        

In [4]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

In [71]:
pprint(final_list_json)

[{'description': '',
  'name': 'BAR BITES',
  'sub_category': [{'description': '',
                    'foodlist': [{'description': 'Crispy Corn Tortillas Topped '
                                                 'With Spicy Baked Bean Salsa '
                                                 'and Sour Cream',
                                  'name': 'NACHOS (Veg/Chicken)',
                                  'price': '190/230'},
                                 {'description': 'Crispy fried Mini Papads '
                                                 'Topped With A Tangy Masala '
                                                 'Mixture And Grated Cheese',
                                  'name': 'CHEESE MASALA PAPAD',
                                  'price': '160'},
                                 {'description': 'A Favorful Version Of '
                                                 'Regular Peanuts Tossed In A '
                                                 'Variety of Spi

In [60]:
for a,b in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
    print(a,b[1])

0 main_category                                            BAR BITES
sub_category                                             BAR BITES
dish_name                                     NACHOS (Veg/Chicken)
description      Crispy Corn Tortillas Topped With Spicy Baked ...
price                                                      190/230
Name: 0, dtype: object
1 main_category                                            BAR BITES
sub_category                                             BAR BITES
dish_name                                      CHEESE MASALA PAPAD
description      Crispy fried Mini Papads Topped With A Tangy M...
price                                                          160
Name: 1, dtype: object
2 main_category                                            BAR BITES
sub_category                                             BAR BITES
dish_name                           H.O.C SIGNATURE MASALA PEANUTS
description      A Favorful Version Of Regular Peanuts Tossed I...
price     

In [9]:
from pprint import pprint

In [210]:
json.load(Restaurant.objects[0].to_json())

AttributeError: 'str' object has no attribute 'read'

In [207]:
with app.app_context():
    
    js=jsonify(final_list_json)

# Taking care of multiple prices

In [ ]:
lengths = str_menu['price'].str.split('/').map(len)

In [ ]:
str_menu = pd.DataFrame({'main_category': np.repeat(str_menu['main_category'], lengths),
                    'sub_category': np.repeat(str_menu['sub_category'], lengths),
                    'name': np.repeat(str_menu['name'],lengths),
                    'description': np.repeat(str_menu['description'],lengths),
                    'price': chainer(str_menu['price'])})

In [ ]:
str_menu.reset_index(inplace=True,drop=True)

In [ ]:
str_menu.loc[45,'price']="250"

In [ ]:
pickle.dump(str_menu,open('half_menu_main.pkl','wb'))

In [ ]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

# js.data

In [17]:
pickle.dump(final_json,open('jason.pkl','wb'))

In [72]:
pickle.dump(final_list_json,open('jason.pkl','wb'))

In [71]:
pickle.dump(my_json,open('jason.pkl','wb'))

NameError: name 'my_json' is not defined

In [122]:
my_json={

  "categories": ["bar_bites", "salads", "veg_starters", "non_veg_starters"],
  "subcategories": {
    "bar_bites" : ["bar_bites1"],
    "salads": ["salads1"],
    "veg_starters": ["veg_starter","from_the_clay_oven"],
    "non_veg_starters":["chicken_starters","from_the_clay_oven","lamb_starters","egg_starters","from_the_grill"]
  },
  "items" :{
    "bar_bites1": [ {
      "dish_name": "NACHOS (Veg/Chicken)",
      "description":
      "Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream",
      "price": "190/230"
    },
      {
        "dish_name": "CHEESE MASALA PAPAD",
        "description":
        "Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese",
        "price": "160"
      },
      {
        "dish_name": "H.O.C SIGNATURE MASALA PEANUTS",
        "description":
        "A Favorful Version Of Regular Peanuts Tossed In A Variety of Spices",
        "price": "160"
      }
    ],
    "salads1":[ {
      "dish_name": "GREEN SALAD",
      "description": "Traditional Green Cuts With Mint Yoghurt Dip",
      "price": "170"
    },
      {
        "dish_name": "HOC SALAD (Veg/Chicken)",
        "description":
        "Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing",
        "price": "180/210"
      }
    ],
    "veg_starter":[{
      "dish_name": "CHEESE & CORN TRIANGLE (new)",
      "description":
      "Shredded Cheese, Corn & Seasonings Wrapped In Wonton Sheet & Deep Fried",
      "price": "220"
    },
      {
        "dish_name": "CHAMPIGNONS FARCIS (new)",
        "description": "Sauteed Button Mushroom Stuffed With Cheese & Herbs",
        "price": "230"
      }
    ],
    "from_the_clay_oven":[{
      "dish_name": "MAKHMALI PANEER TIKKA",
      "description":
      "Mouth Melting Paneer Cubes Marinated With A Cheese Curd Mixture, Cream And Mild Spices",
      "price": "270"
    },
      {
        "dish_name": "ANGARI PANEER TIKKA",
        "description": "Clay Oven Roasted Spicy Morsels Of Cottage Cheese",
        "price": "270"
      }
    ]
  }
}

In [120]:
my_json={

  "categories": ["bar_bites", "salads", "veg_starters", "non_veg_starters"],
  "subcategories": {
    "bar_bites" : ["bar_bites1"],
    "salads": ["salads1"],
    "veg_starters": ["veg_starter","from_the_clay_oven"],
    "non_veg_starters":["chicken_starters","from_the_clay_oven","lamb_starters","egg_starters","from_the_grill"]
  },
  "items" :{
    "bar_bites1": [ {
      "dish_name": "NACHOS (Veg/Chicken)",
      "description":
      "Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream",
      "price": "190/230"
    }]
  }
}

In [ ]:
pages[6]=

In [ ]:
pages[7]=

In [ ]:
pages[8]=

In [ ]:
pages[9]=

In [ ]:
pages[10]=

In [ ]:
pages[11]=